In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import xlrd
import tkinter as tk
from tkinter import filedialog
import math
import glob

In [12]:
def calc_mad(some_data):
    some_data = some_data.loc[:, ['Time', 'X', 'Y', 'Z', 'Magnitude']].dropna()
    all_mad = {}
    time_name = "Time"
        
    # Grab the first timestamp from data
    # print(some_data.shape)
    start = some_data.loc[some_data.index[0], time_name]
    # Specify the amount of time to aggregate over
    agg_len = 5
    # Grab end of aggregation period
    end_time = start + timedelta(seconds=agg_len - 1)
    # print(f"Start Time {start} \nEnd Time {end_time}")
    # Calculate the total length of the trial in seconds
    trial_length = (some_data.loc[some_data.index[-1], time_name] - start).total_seconds()
    # Runs the total length of trial divided by the length of time we aggregate over
    # essentialy creates a window of agg_len, and interval of agg_len
    for i in range(int(trial_length//agg_len)):
        # print(end_time)
        # Get agg_len seconds worth of accelerometer readings
        group_s = some_data.loc[(some_data[time_name] >= start) & (some_data[time_name] <= end_time), :]
        # print(group_s)
        # Get the mean X, Y, and Z of those readings
        agg_s = group_s.aggregate(lambda x : np.mean(x))
        # print(agg_s)
        mag_s = agg_s[4]

        # print(f"{mag_s}")
        # Subtract the mean magnitude from each accelerometer magnitude from each vector magnitude and then take abs
        dif_mean = group_s['Magnitude'].apply(lambda x : abs(x - mag_s))
        # Caclulate the sum of all the vector mags - mean mags. Then divide by the number of vectors
        # print(dif_mean.sum())
        # print(f"Shape of division {dif_mean.shape[0]}")
        # print(dif_mean.shape[0])
        mad = (dif_mean.sum()) / dif_mean.shape[0]

        # print(mad)

        # Add each Mad and the corresponding time to a list :
        all_mad[end_time] = mad
        # 
        start = end_time + timedelta(seconds=1)
        end_time = start + timedelta(seconds=agg_len - 1)
        

    mad_df = pd.Series(data=all_mad, name="MAD")
    return mad_df

In [57]:
align_path = "V:\\R01 - W4K\\2_Shaker project\\Devices Evaluated\\Reliability\\Processed Data\\Round 2\\Apple\\Trials\\"
# align_path = "V:\\R01 - W4K\\2_Shaker project\\Devices Evaluated\\Reliability\\Processed Data\\Round 2\\Actigraph\\Trials\\"
# align_path = "V:\\R01 - W4K\\2_Shaker project\\Devices Evaluated\\Reliability\\Processed Data\\Round 2\\Garmin\\Trials\\"
# align_path = "V:\\R01 - W4K\\2_Shaker project\\Devices Evaluated\\BlackAccelerometerTest\\Processed Data\\Trials\\"
aligned_files = glob.glob(align_path + "Trial [0-9][0-9]*")
agg_files = glob.glob(align_path + "*Agg*")

In [46]:
for align, agg in zip(aligned_files, agg_files):
    # Read in aligned data
    align_data = pd.read_csv(align, parse_dates=['Time'], infer_datetime_format=True)
    
    # Break data up by device id. Then calculate MAD
    ids = align_data['ID'].unique()
    mad = None
    for device in ids:
        id_mad = calc_mad(align_data.loc[(align_data['ID'] == device), ['Time', 'X','Y','Z','Magnitude']])
        id_mad = id_mad.to_frame()
        id_mad.insert(0, "ID", device)
        if mad is None :
            mad = id_mad
        else :
            mad = pd.concat([mad, id_mad])
    old_agg = pd.read_csv(agg, parse_dates=['Time'], infer_datetime_format=True)
    new_agg = old_agg.merge(mad, how='left', left_on=['Time', 'ID'], right_on=[mad.index, 'ID'])
    new_agg.to_csv(agg[:-4] + "_v2.csv", index=None)
    

In [58]:
# Aggregate new files
agg_v2_files = glob.glob(align_path + "*v2.csv")
data = None
for file in agg_v2_files:
    temp = pd.read_csv(file)
    if data is None :
        data = temp
    else :
        data = pd.concat([data,temp])
data.to_csv(align_path + "Aggregated.csv")